<a href="https://colab.research.google.com/github/paulovitorcl/algoritmo-genetico/blob/main/WebP%2BFlip%2BAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método WebP + Flip + AG como solução para ataques adversariais

## 1 - Instalação das dependências

In [ ]:
!pip install numpy pillow tensorflow deap matplotlib
!pip install cleverhans==3.1.0
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00


## 2 - Importação das Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.applications import InceptionV3, ResNet101
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from cleverhans.tf2.attacks import fast_gradient_method
from art.estimators.classification import KerasClassifier
from art.attacks import DeepFool
from deap import base, creator, tools
import os

## 3 - Carregamento do Dataset
O dataset ILSVRC 2012 deve ser baixado e organizado em diretórios.

https://image-net.org/download-images.php

Se você tiver o dataset no seu computador, faça o upload para o Google Drive:

    Acesse o Google Drive.
    Crie uma nova pasta chamada "ILSVRC" (ou outro nome de sua escolha).
    Faça o upload das imagens para essa pasta.

Após esta etapa precisamos montar o Google Driva no Colab:




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Carregando o Dataset

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images_from_folder(folder, max_images=None):
    images = []
    count = 0

    # Percorre todas as subpastas e arquivos
    for subdir, _, files in os.walk(folder):
        for filename in files:
            if filename.endswith(('png', 'jpg', 'jpeg')):  # Adicione formatos de imagem se necessário
                img_path = os.path.join(subdir, filename)
                img = load_img(img_path, target_size=(299, 299))  # Para InceptionV3
                img = img_to_array(img) / 255.0  # Normaliza
                images.append(img)
                count += 1
                # Se max_images for definido, limite o número de imagens carregadas
                if max_images and count >= max_images:
                    return np.array(images)

    return np.array(images)

# Define o caminho do dataset
dataset_path = '/content/drive/MyDrive/images/validation/'  # ajuste o caminho conforme necessário

# Carrega as imagens, limitando a 5000 se necessário
x_test = load_images_from_folder(dataset_path, max_images=5000)

## Caso não queira limitar
#x_test = load_images_from_folder(dataset_path)

## 4 - Definição da função de compressão e flip

In [ ]:
def compress_and_flip(image, qf):
    # Salva a imagem como WebP com a qualidade especificada
    compressed_image_path = 'temp.webp'
    image.save(compressed_image_path, format='WEBP', quality=qf)

    # Recarrega a imagem comprimida
    img_compressed = Image.open(compressed_image_path)

    # Aplica a operação flip
    img_flipped = img_compressed.transpose(Image.FLIP_LEFT_RIGHT)

    return img_flipped


## 5 - Definição do modelo de classificação
Carregue o modelo InceptionV3 ou ResNet101:

In [ ]:
def create_model(model_type='inception'):
    if model_type == 'inception':
        model = InceptionV3(weights='imagenet')
    elif model_type == 'resnet':
        model = ResNet101(weights='imagenet')
    model.trainable = False  # Não treinar o modelo
    return model

model = create_model('inception')  # Ou 'resnet'

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## 6 - Avaliação do modelo com imagens transformadas
Implemente a função que avalia o modelo usando imagens comprimidas e transformadas

In [ ]:
def evaluate_model(model, x_test, qf):
    correct_predictions = 0

    for i in range(len(x_test)):
        original_image = Image.fromarray((x_test[i] * 255).astype(np.uint8))

        # Aplica a compressão e flip
        transformed_image = compress_and_flip(original_image, qf)
        transformed_image = np.array(transformed_image) / 255.0

        # Adiciona a dimensão do batch
        transformed_image = np.expand_dims(transformed_image, axis=0)

        # Faz a previsão
        predictions = model.predict(transformed_image)
        predicted_class = np.argmax(predictions, axis=1)

        # Aqui você deve ter uma forma de obter a classe correta
        correct_classes = ...  # Carregue as classes corretas para x_test
        if predicted_class[0] == correct_classes[i]:
            correct_predictions += 1

    accuracy = correct_predictions / len(x_test)
    return accuracy

## 7 - Avaliação do Modelo sem o algoritmo genético
Avalie a acurácia do modelo com diferentes faixas de QF sem usar o algoritmo genético

In [ ]:
qf_values = [10, 30, 50, 70, 90]
accuracies_no_genetic = []

for qf in qf_values:
    accuracy = evaluate_model(model, x_test, qf)
    accuracies_no_genetic.append(accuracy)
    print(f'Acurácia sem algoritmo genético com QF={qf}: {accuracy:.2f}')

## 8 - Integração do Algoritmo Genético
Adicione a configuração do algoritmo genético para otimizar o QF

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def eval_function(individual):
    qf = int(individual[0])  # O primeiro gene é o QF
    accuracy = evaluate_model(model, x_test, qf)
    return (accuracy,)

toolbox = base.Toolbox()
toolbox.register("QF", np.random.randint, 10, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.QF, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=50, sigma=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Algoritmo genético
population = toolbox.population(n=10)
for gen in range(10):
    fits = list(map(toolbox.evaluate, population))
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    population[:] = offspring

best_ind = tools.selBest(population, 1)[0]
best_qf = int(best_ind[0])
print(f'Best QF found: {best_qf}')


## 9 - Avaliação do Modelo com Algoritmo Genético

In [ ]:
accuracy_genetic = evaluate_model(model, x_test, y_test, best_qf)
print(f'Acurácia com algoritmo genético (QF={best_qf}): {accuracy_genetic:.2f}')

## 10 - Visualizando os resultados da Acurácia
Plote a acurácia em função dos diferentes valores de QF, incluindo a do modelo otimizado pelo algoritmo genético

In [ ]:
# Coletar as acurácias
accuracies_genetic = []
for qf in qf_values:
    accuracy = evaluate_model(model, x_test, qf)
    accuracies_genetic.append(accuracy)

# Visualizando os resultados
plt.figure(figsize=(10, 6))
plt.plot(qf_values, accuracies_no_genetic, marker='o', label='Sem Algoritmo Genético')
plt.axhline(y=accuracy_genetic, color='r', linestyle='--', label=f'Com Algoritmo Genético (QF={best_qf})')
plt.title('Comparação de Acurácia do Modelo')
plt.xlabel('Quality Factor (QF)')
plt.ylabel('Acurácia')
plt.xticks(qf_values)  # Define os ticks do eixo x
plt.grid(True)
plt.legend()
plt.show()


## 11 - Visualização de Imagens Originais e Transformadas

In [ ]:
def display_images(original_images, transformed_images_no_genetic, transformed_images_genetic, num_images=5):
    plt.figure(figsize=(15, 8))

    for i in range(num_images):
        # Imagem original
        plt.subplot(3, num_images, i + 1)
        plt.imshow(original_images[i])
        plt.title("Original")
        plt.axis("off")

        # Imagem transformada sem AG
        plt.subplot(3, num_images, i + 1 + num_images)
        plt.imshow(transformed_images_no_genetic[i])
        plt.title("Transformada (Sem AG)")
        plt.axis("off")

        # Imagem transformada com AG
        plt.subplot(3, num_images, i + 1 + 2*num_images)
        plt.imshow(transformed_images_genetic[i])
        plt.title("Transformada (Com AG)")
        plt.axis("off")

    plt.tight_layout()
    plt.show()


## 12 Aplicação e visualização
Após calcular o QF ótimo com o algoritmo genético, aplique a compressão e flip para ambas as condições e visualize os resultados

In [ ]:
# Supondo que você tenha carregado as imagens em x_test
num_images_to_display = 5
transformed_images_no_genetic = []
transformed_images_genetic = []

# Usando um QF fixo para o caso sem AG
qf_fixed = 50

# Transformações sem algoritmo genético
for i in range(num_images_to_display):
    original_image = Image.fromarray((x_test[i] * 255).astype(np.uint8))
    transformed_image_no_genetic = compress_and_flip(original_image, qf_fixed)
    transformed_images_no_genetic.append(np.array(transformed_image_no_genetic) / 255.0)

# Transformações com algoritmo genético
for i in range(num_images_to_display):
    original_image = Image.fromarray((x_test[i] * 255).astype(np.uint8))
    transformed_image_genetic = compress_and_flip(original_image, best_qf)  # Usando QF otimizado
    transformed_images_genetic.append(np.array(transformed_image_genetic) / 255.0)

# Exibindo as imagens
display_images(x_test[:num_images_to_display], transformed_images_no_genetic, transformed_images_genetic)


## Conclusão

Agora, ao executar o código, você verá três linhas de imagens:

    A primeira linha mostra as imagens originais.
    A segunda linha mostra as imagens transformadas com um QF fixo (sem o algoritmo genético).
    A terceira linha mostra as imagens transformadas com o QF otimizado pelo algoritmo genético.

Isso permitirá que você compare visualmente os efeitos das transformações em ambas as situações.